# Real Estate Data Cleaning (Portugal Listings)

This notebook performs the data cleaning steps as required:
1.  **Load Data**: Import `portugal_listinigs.csv`.
2.  **Remove Duplicates**: Identify and drop duplicate rows.
3.  **Handle Missing Values**: Analyze gaps and impute or drop columns.
4.  **Consistency**: Fix data types and remove invalid rows.
5.  **Save**: Export the clean dataset for analysis.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load Data
file_path = '../7_RAW/portugal_listinigs.csv'
df = pd.read_csv(file_path)

# Display initial info
print(f"Initial Shape: {df.shape}")
df.head()

## 2. Identify and Remove Duplicates
Check for rows that are identical across all columns.

In [ ]:
duplicates = df.duplicated().sum()
print(f"Total Duplicate rRows: {duplicates}")

if duplicates > 0:
    df.drop_duplicates(inplace=True)
    print(f"Duplicates removed. New Shape: {df.shape}")

## 3. Handle Missing Values

In [ ]:
# Calculating percentage of missing values per column
missing_percentage = df.isnull().mean() * 100
missing_df = pd.DataFrame({'Column': df.columns, 'Missing %': missing_percentage})
print(missing_df[missing_df['Missing %'] > 0].sort_values(by='Missing %', ascending=False))

### Strategy for Missing Values:
- **Drop High % Missing**: Columns with >50% missing data (e.g., specific amenities or dates if mostly null).
- **Impute**: Fill numerical gaps (e.g., `Parking` with 0) or categorical (`EnergyCertificate` with 'Unknown').
- **Drop Rows**: Remove rows where critical columns like `Price` are missing.

In [ ]:
# Drop rows where Price is missing (Critical for analysis)
df.dropna(subset=['Price'], inplace=True)

# Impute numerical missing with 0 (e.g., Parking, Garage size)
numeric_cols = ['Parking', 'GrossArea', 'TotalArea', 'ConstructionYear']
for col in numeric_cols:
    if col in df.columns:
        df[col].fillna(0, inplace=True)

# Impute categorical with 'Unknown'
categorical_cols = ['EnergyCertificate', 'Condition']
for col in categorical_cols:
    if col in df.columns:
        df[col].fillna('Unknown', inplace=True)

print(f"Shape after handling missing: {df.shape}")

## 4. Data Consistency

In [ ]:
# Remove rows with unrealistic prices (e.g., 0 or negative)
df = df[df['Price'] > 0]

# Convert ConstructionYear to integer (handling 0s)
if 'ConstructionYear' in df.columns:
    df['ConstructionYear'] = df['ConstructionYear'].astype(int)

# Standardize text (District, City)
text_cols = ['District', 'City', 'Town', 'Type']
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.title()

df.head()

## 5. Save Clean Data

In [ ]:
output_file = '../7_RAW/portugal_listings_cleaned.csv'
df.to_csv(output_file, index=False)
print(f"Cleaned dataset saved to: {output_file}")